In [1]:
#!/data/anaconda3/bin/python
# -*- coding: utf-8 -*-
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import argparse
import time

from pandas import Series, DataFrame

#import model
import random

class Args():
    is_training = False
    layers = 1
    rnn_size = 100  # RNN hidden unit num
    n_epochs = 1  # number of epochs
    batch_size = 512  # default 50
    dropout_p_hidden = 1  # dropout probability in hidden layer
    learning_rate = 0.001
    decay = 0.96  # used for batch norm
    decay_steps = 1000
    sigma = 0  # used for variable initial
    init_as_normal = False
    reset_after_session = True  # wheather resetting the hidden state when a session is finished
    session_key = 'SessionId'  # column 1
    item_key = 'ItemId'  # column 2
    time_key = 'Time'  # column 3
    grad_cap = 0
    test_model = 1  # test model version
    base_path = ''
    checkpoint_dir = ''  # directory of check point
    model_name = ''
    loss = 'cross-entropy'
    final_act = 'softmax'  # activation function of the final layer
    hidden_act = 'tanh'  # activation function of the hidden layer
    n_items = -1  # number of itemes
    ori_file = ''  # ds of data
    rnn_type = ''
    is_preprocess = 0
    DUIEN_layer = 2
    head_num = 2
    predict_sequence_length = 50
    
    attn_item_sbj_hidden = 8
    
    ## keep number of neighbors
    max_degree = 100
    enrich_seq_num = 2


def parseArgs():
    parser = argparse.ArgumentParser(description='DUIEN4Rec args')
    parser.add_argument('--layer', default=1, type=int)  # default for single layer GRU
    parser.add_argument('--size', default=128, type=int)  # rnn size(dimension)
    parser.add_argument('--epoch', default=1, type=int)
    parser.add_argument('--batch_size', default=256, type=int)
    parser.add_argument('--lr', default=0.001, type=float)  # learning rate
    parser.add_argument('--train', default=1, type=int)
    parser.add_argument('--test', default=1, type=int)
    parser.add_argument('--hidden_act', default='tanh', type=str)
    parser.add_argument('--final_act', default='softmax', type=str)
    parser.add_argument('--loss', default='cross-entropy', type=str)
    parser.add_argument('--dropout', default='0.6', type=float)
    parser.add_argument('--test_model', default=0, type=int)
    parser.add_argument('--base_path', default='', type=str)
    parser.add_argument('--ori_file', default='', type=str)
    parser.add_argument('--model_name', default='', type=str)
    parser.add_argument('--rnn_type', default='gru', type=str)
    parser.add_argument('--decay_steps', default=1000, type=int)
    parser.add_argument('--is_preprocess', default=0, type=int)
    parser.add_argument('--predict_sequence_length', default=20, type=int)
    parser.add_argument('--DUIEN_layer', default=2, type=int)
    parser.add_argument('--head_num', default=2, type=int)
    parser.add_argument('--attn_item_sbj_hidden', default=8, type=int)
    parser.add_argument('--sliding_window', default=3, type=int)
    parser.add_argument('--max_degree', default=100, type=int)
    parser.add_argument('--enrich_seq_num', default=2, type=int)
    return parser.parse_args()

C:\Users\VULCAN\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\VULCAN\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\VULCAN\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\VULCAN\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [3]:
from sklearn.preprocessing import normalize

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np
import numpy 

class Classifier(object):

    def __init__(self, vectors, clf):
        self.embeddings = vectors
        self.clf = TopKRanker(clf)
        self.binarizer = MultiLabelBinarizer(sparse_output=True)

    def train(self, X, Y, Y_all):
        self.binarizer.fit(Y_all)
        X_train = [self.embeddings[int(x)] for x in X]
        Y = self.binarizer.transform(Y)
        self.clf.fit(X_train, Y)

    def evaluate(self, X, Y):
        top_k_list = [len(l) for l in Y]
        Y_ = self.predict(X, top_k_list)
        Y = self.binarizer.transform(Y)
        averages = ["micro", "macro", "samples", "weighted"]
        results = {}
        for average in averages:
            results[average] = f1_score(Y, Y_, average=average)
        results['accuracy'] = accuracy_score(Y, Y_)
        
        # print('Results, using embeddings of dimensionality', len(self.embeddings[X[0]]))
        # print('-------------------')
        #print(results)
        return results
        # print('-------------------')

    def predict(self, X, top_k_list):
        X_ = numpy.asarray([self.embeddings[int(x)] for x in X])
        Y = self.clf.predict(X_, top_k_list=top_k_list)
        return Y

    def split_train_evaluate(self, X, Y, train_precent, seed=0):
        state = numpy.random.get_state()

        training_size = int(train_precent * len(X))
        numpy.random.seed(seed)
        shuffle_indices = numpy.random.permutation(numpy.arange(len(X)))
        X_train = [X[shuffle_indices[i]] for i in range(training_size)]
        Y_train = [Y[shuffle_indices[i]] for i in range(training_size)]
        X_test = [X[shuffle_indices[i]] for i in range(training_size, len(X))]
        Y_test = [Y[shuffle_indices[i]] for i in range(training_size, len(X))]

        self.train(X_train, Y_train, Y)
        numpy.random.set_state(state)
        return self.evaluate(X_test, Y_test)


from sklearn.multiclass import OneVsRestClassifier
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        probs = numpy.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            probs_[:] = 0
            probs_[labels] = 1
            all_labels.append(probs_)
        return numpy.asarray(all_labels)
    
from scipy.optimize import linear_sum_assignment as linear_assignment
def acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [4]:
import sys
sys.argv = ['foo']
command_line = parseArgs()       # get external parameters
args = Args()

args.layers = command_line.layer
args.rnn_size = command_line.size
args.n_epochs = command_line.epoch
args.learning_rate = command_line.lr
args.is_training = command_line.train
args.test_model = command_line.test
args.batch_size = command_line.batch_size
args.hidden_act = command_line.hidden_act
args.final_act = command_line.final_act
args.loss = command_line.loss
args.test_model = command_line.epoch - 1  # use the model saved by the last epoch
args.dropout_p_hidden = 1.0 if args.is_training == 0 else command_line.dropout  # use dropout(0.5) in training phase
args.base_path = command_line.base_path
args.ori_file = command_line.ori_file
args.checkpoint_dir = args.base_path + 'checkpoint_foursquare_DUIEN/' + args.ori_file
args.model_name = command_line.model_name
args.rnn_type = command_line.rnn_type
args.decay_steps = command_line.decay_steps
args.is_preprocess = command_line.is_preprocess
args.predict_sequence_length = command_line.predict_sequence_length
args.DUIEN_layer = command_line.DUIEN_layer
args.head_num = command_line.head_num

args.sliding_window = command_line.sliding_window

args.savedModel = 1

args.attn_item_sbj_hidden = command_line.attn_item_sbj_hidden
args.max_degree = command_line.max_degree
args.enrich_seq_num = command_line.enrich_seq_num

In [5]:
args.base_path = "datasets/"
args.ori_file = "foursquare_seq"
#2020052812

In [6]:
if not os.path.exists(args.checkpoint_dir):
        os.mkdir(args.checkpoint_dir)
        with open(args.base_path + args.model_name + ".log", "a") as log_file:
            log_file.write('mkdir ' + args.checkpoint_dir)
else:
    with open(args.base_path + args.model_name + ".log", "a") as log_file:
        log_file.write('already exists: ' + args.checkpoint_dir)

os.environ["CUDA_VISIBLE_DEVICES"]="0"     # specify which GPU(s) to be used
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7, allow_growth=True)    # determines the fraction of the overall amount of memory that each visible GPU can be used

In [7]:
# !/data/anaconda3/bin/python
# -*- coding: utf-8 -*-
import os
import tensorflow as tf
from tensorflow.python.ops import rnn_cell
import pandas as pd
import numpy as np
import time
from pandas import Series, DataFrame
import pickle
from sklearn import metrics
#import layers
import random
# from utils import *
# from rnn import *
import networkx as nx

tf.reset_default_graph()


class DUIEN4Rec:
    def __init__(self, sess, args):
        self.sess = sess
        self.is_preprocess = args.is_preprocess

        self.is_training = args.is_training
        self.layers = args.layers
        self.rnn_size = args.rnn_size
        self.n_epochs = args.n_epochs

        self.dropout_p_hidden = args.dropout_p_hidden
        self.learning_rate = args.learning_rate
        self.decay = args.decay  # default:0.96
        self.decay_steps = args.decay_steps  # default:1e4
        self.sigma = args.sigma  # param for initializer, default:0
        self.init_as_normal = args.init_as_normal  # default:False
        self.reset_after_session = args.reset_after_session  # default:True
        self.session_key = args.session_key
        self.item_key = args.item_key
        self.time_key = args.time_key
        self.grad_cap = args.grad_cap  # default:0
        self.base_path = args.base_path
        self.model_name = args.model_name
        self.rnn_type = args.rnn_type
        self.ori_file = args.ori_file
        self.test_model = args.test_model
        self.sequence_length = args.predict_sequence_length
        self.max_click = 200

        self.DUIEN_layer = args.DUIEN_layer
        self.head_num = args.head_num
        self.um_tag_length = 30
        self.um_sbj_length = 30

        self.train_path = self.base_path
        self.batch_size_pos = args.batch_size
        
        self.attn_item_sbj_hidden = args.attn_item_sbj_hidden
        
        
        self.max_degree = 100
        self.tolerant_time = 40
        
        self.max_text_len = 30
        #self.max_text_len = 20
        
        if self.is_preprocess == 1:
            return

        #self.time_emb_num = 150
        self.time_emb_num = 30
        self.short_emb_size = 4

        if args.hidden_act == 'tanh':
            self.hidden_act = self.tanh
        elif args.hidden_act == 'relu':
            self.hidden_act = self.relu
        else:
            raise NotImplementedError

        # different loss function can try with different final_activation in training phase
        if args.loss == 'cross-entropy':  # point-wise ranking loss
            if args.final_act == 'tanh':
                self.final_activation = self.softmaxth
            else:
                self.final_activation = self.softmax
            self.loss_function = self.cross_entropy
        elif args.loss == 'bpr':  # pair-wise ranking loss
            if args.final_act == 'linear':
                self.final_activation = self.linear
            elif args.final_act == 'relu':
                self.final_activation = self.relu
            else:
                self.final_activation = self.tanh
            self.loss_function = self.bpr
        elif args.loss == 'top1':  # pair-wise ranking loss
            if args.final_act == 'linear':
                self.final_activation = self.linear
            elif args.final_act == 'relu':
                self.final_activatin = self.relu
            else:
                self.final_activation = self.tanh
            self.loss_function = self.top1
        else:
            raise NotImplementedError

        self.checkpoint_dir = args.checkpoint_dir
        if not os.path.isdir(self.checkpoint_dir):
            raise Exception("[!] Checkpoint Dir not found")
    
    def init_model(self):
    
        self.n_items = max(self.ItemIdxList) + 2
        self.n_tags = max(self.TagIdxList) + 2

        self.predictTopN = self.n_items
        # if max(self.ItemIdxList) > 100000:
        #     self.predictTopN = 100000

        self.build_model()

        print("init1")
        if self.is_training == 1:
            self.sess.run(tf.global_variables_initializer())
        print("init2")
        #         self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=10)
        self.saver = tf.train.Saver(max_to_keep=10)
        print("init3")

        if self.is_training == 1:
            return
        
        
        # use self.predict_state to hold hidden states during prediction.
        #         self.predict_state = [np.zeros([self.batch_size, self.rnn_size], dtype=np.float32) for _ in range(self.layers)]
        print("init4")
        ckpt = tf.train.get_checkpoint_state(self.checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            self.saver.restore(self.sess, '{}/DUIEN-model-{}'.format(args.base_path + self.checkpoint_dir, self.test_model))
        else:
            print('no {}/DUIEN-model-{} !!!!!'.format(self.checkpoint_dir, self.test_model))
        print("init5")

    ########################ACTIVATION FUNCTIONS#########################
    def linear(self, X):
        return X

    def tanh(self, X):
        return tf.nn.tanh(X)

    def softmax(self, X):
        return tf.nn.softmax(X)

    def softmaxth(self, X):
        return tf.nn.softmax(tf.tanh(X))

    def relu(self, X):
        return tf.nn.relu(X)

    def sigmoid(self, X):
        return tf.nn.sigmoid(X)

    ############################LOSS FUNCTIONS######################
    def cross_entropy(self, yhat):  # yhat is a tensor with shape of [batch_size, batch_size]
        return tf.reduce_mean(-tf.log(tf.diag_part(yhat) + 1e-24))

    def corss_entropy_2(self, yhat):
        return tf.reduce_mean(-tf.log(yhat[:, 0] + 1e-24))

    def bpr_2(self, yhat):
        yhat_pos = yhat[:, 0]
        yhat_pos_column = tf.reshape(yhat_pos, [-1, 1])
        return tf.reduce_mean(-tf.log(tf.nn.sigmoid(yhat_pos_column - yhat)))

    def bpr(self, yhat):
        yhatT = tf.transpose(yhat)
        return tf.reduce_mean(-tf.log(tf.nn.sigmoid(tf.diag_part(yhat) - yhatT)))

    def top1(self, yhat):
        yhatT = tf.transpose(yhat)
        term1 = tf.reduce_mean(tf.nn.sigmoid(-tf.diag_part(yhat) + yhatT) + tf.nn.sigmoid(yhatT ** 2), axis=0)
        term2 = tf.nn.sigmoid(tf.diag_part(yhat) ** 2) / self.batch_size
        return tf.reduce_mean(term1 - term2)  
    
    def build_transformer(self, layer_num, head_num, model_size, initializer):
        W_TRFM = {}
        for layer in range(layer_num):
            for head in range(head_num):
                # define qkv
                variable_WQ = 'WQ_layer_' + str(layer) + '_head_' + str(head)
                W_TRFM[variable_WQ] = tf.get_variable(variable_WQ, [model_size, model_size / head_num],
                                                      initializer=initializer)
                
                variable_WK = 'WK_layer_' + str(layer) + '_head_' + str(head)
                W_TRFM[variable_WK] = tf.get_variable(variable_WK, [model_size, model_size / head_num],
                                                      initializer=initializer)
                
                variable_WV = 'WV_layer_' + str(layer) + '_head_' + str(head)
                W_TRFM[variable_WV] = tf.get_variable(variable_WV, [model_size, model_size / head_num],
                                                      initializer=initializer)
                
            # define ffn variables
            variable_WFFN = 'WFFN1_layer_' + str(layer)
            W_TRFM[variable_WFFN] = tf.get_variable(variable_WFFN, [model_size, model_size], initializer=initializer)

            variable_BFFN = 'BFFN1_layer_' + str(layer)
            W_TRFM[variable_BFFN] = tf.get_variable(variable_BFFN, [1, 1, model_size], initializer=initializer)

       
        return W_TRFM

    def run_transformer(self, layer_num, head_num, model_size, input_trfm, W_TRFM, h_UM, embedding_tip_weights):
        for layer in range(layer_num):
            for head in range(head_num):
                
                if layer == 0:
                    h_UM_pro = tf.matmul(h_UM, embedding_tip_weights)
                    h_UM_expand = tf.expand_dims(h_UM_pro, 2)
                    attn_item_UM_tmp = tf.nn.leaky_relu(tf.matmul(input_trfm, h_UM_expand))
                    attn_item_UM = tf.nn.softmax(tf.matmul(attn_item_UM_tmp, 
                                            tf.transpose(h_UM_expand, perm=[0, 2, 1]))/ tf.sqrt(tf.cast(self.rnn_size, tf.float32)))
                    input_trfm =  tf.add_n([input_trfm, attn_item_UM])  
                
                attn_Q = tf.nn.relu(
                    tf.tensordot(input_trfm, W_TRFM['WQ_layer_' + str(layer) + '_head_' + str(head)], axes=1))
                attn_K = tf.nn.relu(
                    tf.tensordot(input_trfm, W_TRFM['WK_layer_' + str(layer) + '_head_' + str(head)], axes=1))
                attn_V = tf.nn.relu(
                    tf.tensordot(input_trfm, W_TRFM['WV_layer_' + str(layer) + '_head_' + str(head)], axes=1))
                
#                 tmp_out = tf.matmul(attn_Q, tf.transpose(attn_K, perm=[0, 2, 1])) / tf.sqrt(tf.cast(model_size, tf.float32))
#                 diag_vals = tf.ones_like(tmp_out[0, :, :])
#                 tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() 
#                 masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(tmp_out)[0], 1, 1]) 
#                 paddings = tf.ones_like(masks)*(-2**32+1)
#                 tmp_out = tf.where(tf.equal(masks, 0), paddings, tmp_out)
#                 tmp_out = tf.nn.softmax(tmp_out)
#                 attn_out = tf.matmul(tmp_out, attn_V)
                attn_out = tf.matmul(tf.nn.softmax(
                    tf.matmul(attn_Q, tf.transpose(attn_K, perm=[0, 2, 1])) / tf.sqrt(tf.cast(model_size, tf.float32))),
                    attn_V)
                if head == 0:
                    attn_concat = attn_out
                else:
                    attn_concat = tf.concat([attn_concat, attn_out], 2)

            attn_add = attn_concat + input_trfm
            attn_norm = tf.contrib.layers.layer_norm(attn_add, begin_norm_axis=1)  # [None, input_length, hidden_size]
            attn_ffn = tf.nn.relu(tf.add(tf.tensordot(attn_norm, W_TRFM['WFFN1_layer_' + str(layer)], axes=1),
                                         tf.tile(W_TRFM['BFFN1_layer_' + str(layer)],
                                                 [self.batch_size, self.sequence_length, 1])))

            attn_add2 = attn_norm + attn_ffn
            attn_norm2 = tf.contrib.layers.layer_norm(attn_add2, begin_norm_axis=1)  # [None, input_length, hidden_size]
            
            #input_trfm_tmp = input_trfm
            input_trfm = attn_norm2
            #self.input_trfm = input_trfm
            
#             if layer == 0:
#                 h_UM_pro = tf.matmul(h_UM, embedding_tip_weights)
#                 h_UM_expand = tf.expand_dims(h_UM_pro, 2)
#                 attn_item_UM_tmp = tf.nn.leaky_relu(tf.matmul(input_trfm, h_UM_expand))
#                 attn_item_UM = tf.nn.softmax(tf.matmul(attn_item_UM_tmp, 
#                                         tf.transpose(h_UM_expand, perm=[0, 2, 1]))/ tf.sqrt(tf.cast(self.rnn_size, tf.float32)))
#                 input_trfm =  tf.add_n([input_trfm, attn_item_UM])
#                 #input_trfm = tf.multiply(input_trfm, attn_item_UM)
#                 #input_trfm = tf.add_n([input_trfm, tf.nn.leaky_relu(tf.multiply(input_trfm, attn_item_UM))])
            
             
            
        return input_trfm

    def build_model(self):
        
        # doc id to doc index
        self.item_id_hash_table = tf.contrib.lookup.MutableHashTable(key_dtype=tf.int64, value_dtype=tf.int32,
                                                                     default_value=-1, name="itemIdMap",
                                                                     checkpoint=True)
        self.tag_id_hash_table = tf.contrib.lookup.MutableHashTable(key_dtype=tf.int64, value_dtype=tf.int32, default_value=-1, name="tagIdMap", checkpoint=True)

        # doc index to doc id
        self.item_id_hash_table_reverse = tf.contrib.lookup.MutableHashTable(key_dtype=tf.int64, value_dtype=tf.int64,
                                                                             default_value=-1, name="itemIdMap",
                                                                             checkpoint=True)

        self.input_item_id_tensor = tf.placeholder(tf.int64, [None, None], name='input_item_id_tensor')  # [batch_size, sequence_length]
        self.input_tag_id_tensor = tf.placeholder(tf.int64, [None, None],name='input_tag_id_tensor')  # [batch_size, sequence_length]
 
        self.X_time_diff = tf.placeholder(tf.int32, [None, None], name='input_time_diff')  # [batch_size, sequence_length]

        self.target_item_id_tensor = tf.placeholder(tf.int64, [None], name='target_item_id_tensor')  # [batch_size]

        self.X = self.item_id_hash_table.lookup(self.input_item_id_tensor)
        self.X_tag = self.tag_id_hash_table.lookup(self.input_tag_id_tensor)
        
        self.UM_tip_id_tensor = tf.placeholder(tf.int64, [None,None], name='UM_tip_id_tensor')  
        self.UM_tip_id = self.tag_id_hash_table.lookup(self.UM_tip_id_tensor)

        self.Y = self.item_id_hash_table.lookup(self.target_item_id_tensor)

        
        self.batch_size = tf.placeholder(tf.int32, [], name='batch_size')

        self.global_step = tf.Variable(0, name='global_step', trainable=False)  # global step can not be trained.

        with tf.variable_scope('DUIEN_layer'):
            sigma = self.sigma if self.sigma != 0 else np.sqrt(6.0 / (self.n_items + self.rnn_size))
            if self.init_as_normal:
                initializer = tf.random_normal_initializer(mean=0, stddev=sigma)
            else:
                initializer = tf.random_uniform_initializer(minval=-sigma, maxval=sigma)  # for default

            embedding = tf.get_variable('embedding', [self.n_items, self.rnn_size], initializer=initializer)
            embedding_tag = tf.get_variable('embedding_tag', [self.n_tags, self.rnn_size], initializer=initializer)
         
            embedding_tip_weights = tf.get_variable('embedding_tip_weights', [self.rnn_size, self.rnn_size], initializer=initializer)
            
            #um_W = tf.get_variable('um_w', [self.rnn_size, self.rnn_size], initializer=initializer)
            #um_b = tf.get_variable('um_b', [self.rnn_size], initializer=initializer)        
            
            embedding_pos = tf.get_variable('embedding_pos', [1, self.max_click, self.rnn_size], initializer=initializer)

            feat_W = tf.get_variable('feat_w', [self.rnn_size * 2, self.rnn_size], initializer=initializer)
            feat_b = tf.get_variable('feat_b', [self.rnn_size], initializer=initializer)
            
            text_W = tf.get_variable('text_W', [self.max_text_len, 1], initializer=initializer)
            text_b = tf.get_variable('text_b', [1], initializer=initializer)
            
            um_W = tf.get_variable('um_W', [self.max_text_len, 1], initializer=initializer)
            um_b = tf.get_variable('um_b', [1], initializer=initializer)
            
            behavior_um_W = tf.get_variable('behavior_um_W', [self.rnn_size * 2, self.rnn_size], initializer=initializer)
            behavior_um_b = tf.get_variable('behavior_um_b', [self.rnn_size], initializer=initializer)

            user_W = tf.get_variable('user_w', [self.rnn_size * 3, self.rnn_size ], initializer=initializer)
            user_b = tf.get_variable('user_b', [self.rnn_size ], initializer=initializer)

            embedding_time_diff = tf.get_variable('embedding_time_diff', [self.time_emb_num, self.rnn_size], initializer=initializer)

            self.softmax_W = tf.get_variable('softmax_w', [self.n_items, self.rnn_size], initializer=initializer)
            self.softmax_b = tf.get_variable('softmax_b', [self.n_items], initializer=tf.constant_initializer(0.0))

            self.W_TRFM = self.build_transformer(self.DUIEN_layer, self.head_num, self.rnn_size, initializer)

            #item_tag_W = tf.get_variable('item_tag_W', [self.rnn_size * 2, self.rnn_size ], initializer=initializer)
            #item_tag_b = tf.get_variable('item_tag_b', [self.rnn_size ], initializer=initializer)
            
            ### tip conv
            self.UM_tag_emb = tf.nn.embedding_lookup(embedding_tag, self.UM_tip_id + 1)
            self.UM_tag_emb_re = tf.reshape(self.UM_tag_emb,[-1,self.max_text_len,self.rnn_size])
            self.UM_tag_emb_re = tf.expand_dims(self.UM_tag_emb_re, -1)
            
            self.UM_W = tf.Variable(tf.truncated_normal([3, 10, 1, 1], stddev=0.3))
            self.conv_UM = tf.nn.conv2d(self.UM_tag_emb_re, self.UM_W, strides=[1, 1, 1, 1], padding='SAME')
            self.h_UM = self.conv_UM
            self.h_UM = tf.squeeze(self.h_UM, -1)
            self.h_UM = tf.transpose(self.h_UM, perm=[0, 2, 1])
            self.h_UM = tf.matmul(self.h_UM, um_W)
            self.h_UM = tf.add(self.h_UM, um_b)
            
            self.h_UM = tf.squeeze(self.h_UM, -1)
            self.h_UM = tf.nn.tanh(self.h_UM)
            
            ## tag conv
            #self.text_tag_embed = tf.Variable(tf.truncated_normal([len(self.TagIdxList), self.rnn_size], stddev=0.3))
            #self.Text_a = tf.placeholder(tf.int32, [None, self.max_text_len], name='Ta')
            #self.TA = tf.nn.embedding_lookup(self.text_tag_embed, self.Text_a)
            
            #self.X_tag = tf.reshape(self.X_tag,[-1,self.max_text_len])
            self.TA = tf.nn.embedding_lookup(embedding_tag, self.X_tag + 1)
            
            #self.T_A = tf.expand_dims(self.TA, -1)
            self.T_A = tf.reshape(self.TA,[-1,self.sequence_length,self.max_text_len,self.rnn_size])
            
            #self.T_A = self.TA
            #self.W2 = tf.Variable(tf.truncated_normal([2, self.rnn_size, 1, 1], stddev=0.3))
            self.W2 = tf.Variable(tf.truncated_normal([20, 5, self.rnn_size, self.rnn_size], stddev=0.3))
            self.convA = tf.nn.conv2d(self.T_A, self.W2, strides=[1, 1, 1, 1], padding='SAME')
            #self.convA = tf.nn.leaky_relu(self.convA)
            #self.hA = tf.nn.tanh(self.convA)
            #self.hA = tf.nn.tanh(tf.squeeze(self.convA, -1))
            
            #self.hA = tf.squeeze(self.convA, -1)
            self.hA = self.convA
            #self.hA = tf.reshape(self.hA,[-1,self.sequence_length,self.max_text_len,self.rnn_size])
            #self.hA = tf.nn.tanh(self.hA)
            
            self.hA = tf.transpose(self.hA, perm=[0, 1, 3, 2])
            self.hA = tf.matmul(self.hA, text_W)
            self.hA = tf.add(self.hA, text_b)
            
            self.hA = tf.squeeze(self.hA, -1)
            self.hA = tf.nn.tanh(self.hA)
            ###
            
            
            ### input process
            self.inputs_item = tf.nn.embedding_lookup(embedding, self.X + 1)  # [batch_size, sequence_length, rnn_size]
            #inputs_tag = tf.nn.embedding_lookup(embedding_tag, self.X_tag + 1)  #

            # [batch_size, sequence_length, rnn_size * 2]
            #inputs_feat = tf.concat([self.inputs_item, inputs_tag], 2)
            inputs_feat = tf.concat([self.inputs_item, self.hA], 2)
            #inputs_feat = tf.concat([self.inputs_item, self.inputs_item], 2)
            #inputs_feat = tf.nn.leaky_relu(tf.add(tf.matmul(inputs_feat, item_tag_W), item_tag_b)) 
            #inputs_feat = self.inputs_item
            
            # [batch_size, sequence_length, rnn_size]
            self.inputs_time_diff = tf.nn.embedding_lookup(embedding_time_diff, self.X_time_diff + 1)
            self.inputs_tmp = tf.nn.tanh( tf.add(tf.tensordot(inputs_feat, feat_W, axes=1), feat_b)) + self.inputs_time_diff

            self.inputs = tf.reshape(self.inputs_tmp, [self.batch_size, -1, self.rnn_size])

            embedding_pos_slice = tf.slice(embedding_pos, [0, 0, 0], [1, self.sequence_length, self.rnn_size])
            embedding_pos_rev = tf.reverse(embedding_pos_slice, [1])

            self.inputs_pe = tf.add(self.inputs, tf.tile(embedding_pos_rev, [self.batch_size, 1, 1]))

            self.trfm_input_um = self.inputs_pe

            self.trfm_out  = self.run_transformer(self.DUIEN_layer, self.head_num, self.rnn_size, self.trfm_input_um, self.W_TRFM, self.h_UM, embedding_tip_weights)
            #self.trfm_tip_out = tf.concat([self.trfm_out, self.h_UM], 2)
            #self.trfm_out = tf.nn.leaky_relu(tf.add(tf.matmul(self.trfm_tip_out, behavior_um_W), behavior_um_b))
            #self.trfm_out   = tf.nn.dropout(self.trfm_out, keep_prob=self.dropout_p_hidden)
            
            ## attn user interest
            #self.tmp_trfm_out = self.trfm_out
#             self.h_UM_pro = tf.matmul(self.h_UM, embedding_tip_weights)
#             self.h_UM_expand = tf.expand_dims(self.h_UM_pro, 2)
#             #self.attn_item_UM_tmp = tf.nn.leaky_relu(tf.matmul(self.trfm_out, self.h_UM_expand), alpha=0.2)
#             self.attn_item_UM_tmp = tf.nn.leaky_relu(tf.matmul(self.trfm_out, self.h_UM_expand))
#             self.attn_item_UM = tf.nn.softmax(tf.matmul(self.attn_item_UM_tmp, 
#                                     tf.transpose(self.h_UM_expand, perm=[0, 2, 1]))/ tf.sqrt(tf.cast(self.rnn_size, tf.float32)))
#             self.trfm_out =  tf.add_n([self.trfm_out, self.attn_item_UM])
            ###
            
            
            
            self.trfm_final = tf.slice(self.trfm_out, [0, self.sequence_length - 2, 0], [self.batch_size, 2, self.rnn_size])
            self.final_state = tf.reshape(self.trfm_final, [self.batch_size, -1])
            # self.final_state = tf.squeeze(self.trfm_final)

        self.user_vec1 = tf.reshape(self.final_state, [self.batch_size, -1])
        
        ### user concat
        self.user_vec1 = tf.concat([self.user_vec1, self.h_UM], 1)
        #self.user_final = tf.nn.leaky_relu(tf.add(tf.matmul(self.user_vec1, user_W), user_b))
        #self.user_final = tf.add(tf.matmul(self.user_vec1, user_W), user_b)
        self.user_final = tf.matmul(self.user_vec1, user_W)

        if self.is_training == 1:
            sampled_W = tf.nn.embedding_lookup(self.softmax_W, self.Y + 1)
            sampled_b = tf.nn.embedding_lookup(self.softmax_b, self.Y + 1)
            self.logits = tf.matmul(self.user_final, sampled_W, transpose_b=True) + sampled_b
            self.yhat = self.final_activation(self.logits)
            self.cost = self.loss_function(self.yhat)
        #             self.prob = tf.nn.sigmoid(self.yhat)
        else:
            softmax_W_topN = tf.slice(self.softmax_W, [0, 0], [self.predictTopN, self.rnn_size])
            softmax_b_topN = tf.slice(self.softmax_b, [0], [self.predictTopN])
            
            self.logits = tf.matmul(self.user_final, softmax_W_topN, transpose_b=True) + softmax_b_topN
            # self.logits = tf.matmul(self.user_vec2, softmax_W, transpose_b=True) + softmax_b
            self.yhat = self.final_activation(self.logits)
            
            # ## savedModel ## #
            self.yhat = tf.identity(self.yhat, name="yhat")

            # topNum = tf.divide(tf.constant(1000), tf.max(tf.constant(1), outSeqNum))
            self.topk_prob_sample, self.topk_ind_sample = tf.nn.top_k(self.yhat, 50, name="topk")

            self.topk_ind_sample = self.topk_ind_sample - 1

            self.topk_doc_id = self.item_id_hash_table_reverse.lookup(tf.cast(self.topk_ind_sample, tf.int64))

            topk_prob_sample_1d = tf.reshape(self.topk_prob_sample, [-1])
            topk_ind_sample_1d = tf.reshape(self.topk_ind_sample, [-1])
            topk_doc_id_1d = tf.reshape(self.topk_doc_id, [-1])

            self.topk_ind_val = tf.add(topk_ind_sample_1d, tf.constant(0), name="topk_ind_val")
            self.topk_prob_val = tf.add(topk_prob_sample_1d, tf.constant(0.0), name="topk_prob_val")
            self.topk_doc_id = tf.identity(topk_doc_id_1d, name="topk_doc_id")
            return

        self.lr = tf.maximum(1e-5, tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_steps,
                                                              self.decay, staircase=True))

        ''' Try different optimizers. '''
        # optimizer = tf.train.AdagradOptimizer(self.lr)
        # optimizer = tf.train.AdadeltaOptimizer(self.lr)
        # optimizer = tf.train.RMSPropOptimizer(self.lr)
        optimizer = tf.train.AdamOptimizer(self.lr)

        tvars = tf.trainable_variables()
        gvs = optimizer.compute_gradients(self.cost, tvars)
        if self.grad_cap > 0:
            capped_gvs = [(tf.clip_by_norm(grad, self.grad_cap), var) for grad, var in gvs]
        else:
            capped_gvs = gvs
        self.train_op = optimizer.apply_gradients(capped_gvs, global_step=self.global_step)
        
    def save_obj(self, path, obj, name):
        with open(path + name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    def load_obj(self, path, name):
        with open(path + name + '.pkl', 'rb') as f:
            return pickle.load(f)
            
    def preprocess(self, verbose=False):

        
        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write( "{} begin preprocessing...".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())) + '\n')

        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('start loading file: train_file: ' + self.train_path + self.ori_file + '\n')

        #         try:
        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('loading raw data. \n')
            
        

        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('finish loading raw data. \n')


        # get expand data
        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('getting expand data. \n')
        
        mv_data_pd = pd.read_pickle(self.base_path + self.ori_file+".pkl")
        mv_data_pd['Sequence'] = mv_data_pd['Sequence'].astype('str')
        #data_expand = pd.concat([Series(row['SessionId'], row['Sequence'].split('#')) for _, row in
        #                         mv_data_pd.iterrows()]).reset_index()
                                 
        #data_expand.columns = ['ItemInfo', 'SessionId']
        data_expand1 = pd.concat([Series(row['SessionId'], row['Sequence'].split('#')) 
                         for _, row in mv_data_pd.iterrows()]).reset_index()
        data_expand2 = pd.concat([Series(row['tips'], row['Sequence'].split('#')) 
                                 for _, row in mv_data_pd.iterrows()]).reset_index()
        
        data_expand1.columns = ['ItemInfo', 'SessionId']
        data_expand2.columns = ['ItemInfo', 'tips']
        
        #data_expand = pd.merge(data_expand1, data_expand2, on='ItemInfo', how='inner')
        #data_expand.columns = ['ItemInfo', 'SessionId', 'tips']
        data_expand = pd.concat([data_expand1, data_expand2], axis=1).reindex(data_expand1.index)
        data_expand.columns = ['ItemInfo', 'SessionId', 'ItemInfo1', 'tips']
        data_expand.drop(['ItemInfo1'], axis=1)
        
        data_expand[['ItemId', 'Time' ]] = pd.DataFrame( data_expand.ItemInfo.str.split('_').tolist())

        # get item, tag, subject id-idx dictionary dataframe
        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('getting item, tag, subject id-idx dictionary. \n')

        ItemIdCount = data_expand['ItemId'].value_counts()
        ItemIdFreq = ItemIdCount.keys().tolist()
        ItemIdFreqPd = pd.DataFrame(ItemIdFreq)
        ItemIdFreqPd.columns = ['ItemId']
        ItemIdPd = ItemIdFreqPd
        ItemIdPd['ItemIdx'] = range(len(ItemIdFreq))
        ItemIdPd.columns = ['ItemId', 'ItemIdx']

        data_ItemInfo = data_expand[['ItemId']]
        data_ItemInfo.drop_duplicates(inplace=True)
        # data_ItemInfo = pd.merge(data_ItemInfo, ItemIdFreqPd, on='ItemId', how='inner')

        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('getting feature data. \n')
            
            
        # get feature data
        data_ItemInfo.dropna(inplace=True)
        
        data = data_expand[['SessionId', 'ItemId', 'Time', 'tips']]
        data = pd.merge(data, data_ItemInfo, on='ItemId', how='inner')
        
        # add tag info
        mv_data_tag_pd = pd.read_pickle(self.base_path + self.ori_file + "_tags.pkl")
        data = pd.merge(data, mv_data_tag_pd, on='ItemId', how='inner')
        
        
            
        data = data[data['ItemId'] != '']
        data = data[data['Time'] != '']
        
        data.ItemId = data.ItemId.astype(int)
        data.Time = data.Time.astype(int)
        data = data.sort_values(by=['SessionId', 'Time'])

        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('getting offset_sessions. \n')

        # get offset_sessions
        offset_sessions = np.zeros(data['SessionId'].nunique() + 1, dtype=np.int32)
        offset_sessions[1:] = data.groupby('SessionId').size().cumsum()

        with open(self.base_path + self.model_name + ".log", "a") as log_file:
            log_file.write('saving data by pickle. \n')

        # save obj
        
        self.save_obj(self.train_path, offset_sessions, self.ori_file + '_offset_sessions')
        self.save_obj(self.train_path, data, self.ori_file + '_data')
        self.save_obj(self.train_path, ItemIdPd, self.ori_file + '_ItemIdPd')
        
    
    def enrich_edges(self):
        G_item_item = nx.read_edgelist(self.train_path + self.ori_file + '_item_item_edge_list.txt', nodetype=int)
        G_tag_tag = nx.read_edgelist(self.train_path + self.ori_file + '_tag_tag_edge_list.txt', nodetype=int)
        G_item_tag = nx.read_edgelist(self.train_path + self.ori_file + '_item_tag_edge_list.txt', nodetype=int)
        
        data = self.load_obj(self.train_path, self.ori_file + '_data')
        
        Set_item_id = set(data['ItemId'].tolist())
        Set_item_idx = {}
        Set_item_idx_reverse = {}
        Set_tag_id = set(data['TagId'].tolist())
        Set_tag_idx = {}
        Set_tag_idx_reverse = {}

        for i,j in enumerate(Set_item_id):
            Set_item_idx[j] = i
            Set_item_idx_reverse[i] = j
        for i,j in enumerate(Set_tag_id):
            Set_tag_idx[j] = i
            Set_tag_idx_reverse[i] = j
            
        seq_adj_item_item = np.ones((len(Set_item_id), self.max_degree))
        for node_id in G_item_item.nodes():
            neighbors = np.array( [Set_item_idx[i] for i in list(G_item_item.neighbors(node_id))] )
            if len(neighbors) > self.max_degree:
                neighbors = np.random.choice(neighbors, self.max_degree, replace=False)
            elif len(neighbors) < self.max_degree:
                neighbors = np.random.choice(neighbors, self.max_degree, replace=True)
            seq_adj_item_item[Set_item_idx[node_id],:] = neighbors
            
        seq_adj_tag_tag = np.ones((len(Set_tag_id), self.max_degree))
        for node_id in G_tag_tag.nodes():
            neighbors = np.array( [Set_tag_idx[i] for i in list(G_tag_tag.neighbors(node_id))] )
            
            if len(neighbors) > self.max_degree:
                neighbors = np.random.choice(neighbors, self.max_degree, replace=False)
            elif len(neighbors) < self.max_degree:
                neighbors = np.random.choice(neighbors, self.max_degree, replace=True)
            seq_adj_tag_tag[Set_tag_idx[node_id],:] = neighbors
            
        seq_adj_item_tag = np.ones((len(Set_item_id), DUIEN.max_degree))
        for node_id in G_item_item.nodes():
            neighbors = np.array( [Set_tag_idx[int(i.split("_")[0])] for i in list(G_item_tag.neighbors(str(node_id)))] )
            if len(neighbors) > DUIEN.max_degree:
                neighbors = np.random.choice(neighbors, DUIEN.max_degree, replace=False)
            elif len(neighbors) < DUIEN.max_degree:
                neighbors = np.random.choice(neighbors, DUIEN.max_degree, replace=True)
            seq_adj_item_tag[Set_item_idx[node_id],:] = neighbors
            
        seq_adj_tag_item = np.ones((len(Set_tag_id), DUIEN.max_degree))
        for node_id in G_tag_tag.nodes():
            neighbors = np.array( [Set_item_idx[int(i)] for i in list(G_item_tag.neighbors(str(node_id)+"_tag"))] )
            if len(neighbors) > DUIEN.max_degree:
                neighbors = np.random.choice(neighbors, DUIEN.max_degree, replace=False)
            elif len(neighbors) < DUIEN.max_degree:
                neighbors = np.random.choice(neighbors, DUIEN.max_degree, replace=True)
            seq_adj_tag_item[Set_tag_idx[node_id],:] = neighbors
          

        
            
        with open(self.base_path + self.model_name + ".log", "a") as log_file:
                log_file.write('enrich meta path edgelists. \n')
                
        print('enrich meta path edgelists')
                
        with open(self.train_path + self.ori_file + '_item_item_edge_list.txt', 'a') as f:
            for item in G_item_item.nodes():
                for item_i in range(G_item_item.degree(item)):
                    random_tag_id = Set_tag_idx_reverse[random.choice(seq_adj_item_tag[Set_item_idx[item]])]
                    random_item_id = Set_item_idx_reverse[random.choice(seq_adj_tag_item[Set_tag_idx[random_tag_id]] )]
                    f.write("{} {}\n".format(str(item_i),str(random_item_id)))
            f.close()
        
        with open(self.train_path + self.ori_file + '_tag_tag_edge_list.txt', 'a') as f:
            for tag in G_tag_tag.nodes():
                for tag_i in range(G_tag_tag.degree(tag)):
                    random_item_id = Set_item_idx_reverse[random.choice(seq_adj_tag_item[Set_tag_idx[tag]] )]
                    random_tag_id = Set_tag_idx_reverse[random.choice(seq_adj_item_tag[Set_item_idx[random_item_id]] )]
                    f.write("{} {}\n".format(str(tag),str(random_tag_id)))
            f.close()
            
        

    def loaddata(self, verbose=False):
        # load obj
        print('loading data by pickle. \n')

        train_path = self.train_path
        ori_file = self.ori_file
        self.offset_sessions = self.load_obj(train_path, ori_file + '_offset_sessions')
        self.data = self.load_obj(train_path, ori_file + '_data')
        self.ItemIdPd = self.load_obj(train_path, ori_file + '_ItemIdPd')

        self.ItemIdPd = self.ItemIdPd[self.ItemIdPd['ItemId'] != '']

        self.ItemIdPd.dropna(inplace=True)
        self.data.dropna(inplace=True)

        self.ItemIdPd = self.ItemIdPd[self.ItemIdPd['ItemId'] != 'nan']

        self.ItemIdList = [int(k) for k in self.ItemIdPd.ItemId.tolist()]
        self.ItemIdxList = [int(k) for k in self.ItemIdPd.ItemIdx.tolist()]

        tags_set = set()
        for tags in self.data.tags.tolist():
            for i in tags.split(' '):
                tags_set.add(i)
        self.TagIdxList = range(len(tags_set))
        

        #test_cut = int(self.data.shape[0] * 1 / 2)
        #self.data = self.data.iloc[test_cut:-1]
        #self.data_test = self.data.iloc[0:test_cut]

    def toFloat(self, inVal):
        try:
            outVal = float(inVal)
        except:
            outVal = 0.0

        if np.isnan(outVal):
            return 0.0
        else:
            return outVal
    def test(self):
        
        def sigmoid(input_val):
            return 1 / (1 + (math.e ** -input_val))

        self.error_during_train = False
        self.sess.run(self.item_id_hash_table.insert(self.ItemIdList, self.ItemIdxList))
        self.sess.run(self.item_id_hash_table_reverse.insert(self.ItemIdxList, self.ItemIdList))
        
        
        max_test_auc = 0

        self.offset_sessions = np.zeros(self.data['SessionId'].nunique() + 1, dtype=np.int32)
        self.offset_sessions[1:] = self.data.groupby('SessionId').size().cumsum()

        cost_train_list = []
        cost_test_list = []

        auc_train_list = []
        auc_test_list = []

        session_idx_arr = np.arange(len(self.offset_sessions) - 2)
        
        self.HR_10  = 0
        self.HR_5  = 0
        self.HR_1  = 0
        
        self.MRR = 0
        
        for s_idx in session_idx_arr:
            start = self.offset_sessions[s_idx]
            end = self.offset_sessions[s_idx + 1]
            if end - start > 1:
                #pos_idx = end - 1
                if start == end-2:
                    pos_idx = [start]
                else:
                    pos_idx = np.arange(start, end-2)
                
                self.in_idx = np.array(self.data.ItemId.values[pos_idx].tolist())
                self.in_tag = self.data.tags.values[pos_idx].tolist()
                self.in_tag_list = np.asarray([in_tag_i.split(' ') for in_tag_i in self.in_tag])
                self.in_tag = self.in_tag_list.astype(np.int)
                
                self.out_idx = np.array(self.data.ItemId.values[end-1])
                #self.out_idx = np.array(np.ones((len(pos_idx))) * self.data.ItemId.values[end])
                
                
                self.in_um_tip_id_list = [self.data['tips'][posid] for posid in pos_idx]
                self.in_um_tip_id = np.asarray([in_tip_i.split(' ') for in_tip_i in self.in_um_tip_id_list])[0].reshape((1,-1))
                self.in_um_tip_id = self.in_um_tip_id.astype(np.int)
                
                self.in_time = self.data.Time.values[pos_idx].tolist()
                self.out_time = (np.ones((len(pos_idx))) * self.data.Time.values[end]).tolist()
                
                #self.in_time_diff1 = [self.out_time - self.in_time]
                self.in_time_diff1 = [self.out_time[k] - self.in_time[k] for k in range(len(self.out_time))]
                self.in_time_diff2 = [int(k % 30) for k in self.in_time_diff1]
                self.in_time_diff3 = [min(28, k) for k in self.in_time_diff2]
                self.in_time_diff = [max(0, k) for k in self.in_time_diff3]
                
                #self.in_idx_matrix = np.reshape(np.array(in_idx), [-1, 1])
                #self.in_time_diff_matrix = np.reshape(np.array(self.in_time_diff), [-1, 1]) 
            
                self.in_idx_matrix = np.zeros([1, self.sequence_length], dtype=int)
                self.in_time_diff_matrix = np.zeros([1, self.sequence_length], dtype=int) + 28
                
                self.in_idx_matrix = np.concatenate([np.reshape(np.array(self.in_idx), [1, -1]), self.in_idx_matrix[:, 0:-1]], 1)
                self.in_time_diff_matrix = np.concatenate([np.reshape(np.array(self.in_time_diff),
                                                                     [1, -1]), self.in_time_diff_matrix[:, 0:-1]], 1)
                
                self.in_idx_matrix = self.in_idx_matrix[:,0:self.sequence_length]
                self.in_time_diff_matrix = self.in_time_diff_matrix[:,0:self.sequence_length]
                
                self.in_tag_matrix = np.zeros([1, self.sequence_length * self.max_text_len], dtype=int)
                
                self.max_top_tag = 20
                if self.in_tag.shape[0] > self.max_top_tag:
                    self.top_tag = 20
                else:
                    self.top_tag = self.in_tag.shape[0]
                
                self.in_tag_matrix = np.concatenate([np.reshape(np.array(self.in_tag[0:self.top_tag,:]), [1, -1]), 
                                                     self.in_tag_matrix[:,0:-(self.top_tag*self.max_text_len)]], 1)
                
                def flip_swap_zero(matrix_in, fillval=-1):
                        matrix_2d = []
                        for batch_cnt in range(matrix_in.shape[0]):
                            tmp_val = matrix_in[batch_cnt]
                            tmp_val = np.flip(tmp_val[np.nonzero(tmp_val - fillval)], 0)
                            matrix_2d.append(np.concatenate(
                                [tmp_val, np.array([fillval] * (len(matrix_in[batch_cnt]) - len(tmp_val)))], 0))
                        return matrix_2d

                self.in_idx_matrix_swap = flip_swap_zero(self.in_idx_matrix)
                self.in_time_diff_matrix_swap = flip_swap_zero(self.in_time_diff_matrix, fillval=28)
                
                self.in_tag_matrix_swap = flip_swap_zero(self.in_tag_matrix)

        
                feed_dict_test = {self.input_item_id_tensor: np.nan_to_num(self.in_idx_matrix_swap),
                                  self.input_tag_id_tensor: np.nan_to_num(self.in_tag_matrix_swap),
                                  self.X_time_diff: np.nan_to_num(self.in_time_diff_matrix_swap),
                                  self.UM_tip_id_tensor: np.nan_to_num(np.array(self.in_um_tip_id)),
                                  #self.target_item_id_tensor: [np.nan_to_num(np.array(self.out_idx))],

                                  
                                  self.batch_size: 1
                                  }
                
                
                fetches_test = [self._topk_ind_val, self._topk_prob_val, self._topk_doc_id]
                self.topk_ind, self.topk_prob, self.topk_doc = self.sess.run(fetches_test, feed_dict_test)
                
                if self.out_idx  in self.topk_doc[0:10]:
                    self.HR_10 += 1
                    
                if self.out_idx  in self.topk_doc[0:5]:
                    self.HR_5 += 1
                    
                if self.out_idx  in self.topk_doc[0:1]:
                    self.HR_1 += 1
                    
                if self.out_idx  in self.topk_doc:
                    self.MRR += 1/ (list(self.topk_doc).index(self.out_idx ) + 1)
                    
        print("HR_10 ",self.HR_10 / len(self.offset_sessions))
        print("HR_5 ",self.HR_5 / len(self.offset_sessions))
        print("HR_1 ",self.HR_1 / len(self.offset_sessions))   
        print("MRR_10", self.MRR / len(self.offset_sessions))   
 
                
    def fit(self, verbose=False):

        def sigmoid(input_val):
            return 1 / (1 + (math.e ** -input_val))

        self.error_during_train = False
        self.sess.run(self.item_id_hash_table.insert(self.ItemIdList, self.ItemIdxList))
        self.sess.run(self.item_id_hash_table_reverse.insert(self.ItemIdxList, self.ItemIdList))
        
        
        max_test_auc = 0
        
        for epoch in range(self.n_epochs):

            

            self.offset_sessions = np.zeros(self.data['SessionId'].nunique() + 1, dtype=np.int32)
            self.offset_sessions[1:] = self.data.groupby('SessionId').size().cumsum()

            cost_train_list = []
            cost_test_list = []
            cut_cost_test_list = []

            auc_train_list = []
            auc_test_list = []
            cut_auc_test_list = []
            
            session_idx_arr = np.arange(len(self.offset_sessions) - 1)
                
            iters = np.arange(self.batch_size_pos)
            maxiter = iters.max()  # self.batch_size - 1
            # iters is a array, so start will return a array with size of batch_size
            start = self.offset_sessions[session_idx_arr[iters]]
            end = self.offset_sessions[session_idx_arr[iters] + 1] 
            
            
            finished = False
            
            tolerant_time = self.tolerant_time

            train_global_step = 0
            step = 0

            in_idx_matrix = np.zeros([self.batch_size_pos, self.sequence_length], dtype=int)
            in_tag_matrix = np.zeros([self.batch_size_pos, self.sequence_length * self.max_text_len], dtype=int)
            in_time_diff_matrix = np.zeros([self.batch_size_pos, self.sequence_length], dtype=int) + 28

            while not finished:
                minlen = (end - start).min()
                
                out_idx = self.data.ItemId.values[start]
                #out_tag = self.data.TagId.values[start]
                out_sessionId = self.data.SessionId.values[start]
                out_time = self.data.Time.values[start]
                
                ###
                for i in range(minlen - 1):

                    pos_idx = start + i
                    in_idx = self.data.ItemId.values[pos_idx].tolist()  # * (1 + self.neg_pos_sam_rat)
                    in_tag = self.data.tags.values[pos_idx].tolist()  # * (1 + self.neg_pos_sam_rat)
                    
                    
                    in_tag_list = np.asarray([in_tag_i.split(' ') for in_tag_i in in_tag])
                    in_tag = in_tag_list.astype(np.int)
                    
                    in_sessionId = self.data.SessionId.values[pos_idx].tolist()  # * (1 + self.neg_pos_sam_rat)
                    
#                     print("in_idx", in_idx)
#                     print("pos_idx", pos_idx)
#                     print("in_sessionId", in_sessionId)
#                     break
                    
                    in_um_tip_id_list = [self.data['tips'][posid] for posid in pos_idx]
                    in_um_tip_id = np.asarray([in_tip_i.split(' ') for in_tip_i in in_um_tip_id_list])
                    in_um_tip_id = in_um_tip_id.astype(np.int)
                    #self.in_um_tip_id = in_um_tip_id

                    train_idx = start + i + 1
                    out_idx = self.data.ItemId.values[train_idx]

                    # build time diff
                    self.in_time = self.data.Time.values[pos_idx].tolist()
                    self.out_time = self.data.Time.values[train_idx].tolist()
                    
                    self.in_time_diff1 = [self.out_time[k] - self.in_time[k] for k in range(len(self.out_time))]
                    self.in_time_diff2 = [int(k % 30) for k in self.in_time_diff1]
                    self.in_time_diff3 = [min(28, k) for k in self.in_time_diff2]
                    self.in_time_diff = [max(0, k) for k in self.in_time_diff3]
                
                   

                    in_idx_matrix = np.concatenate([np.reshape(np.array(in_idx), [-1, 1]), in_idx_matrix[:, 0:-1]], 1)
                    
                    self.in_tag = in_tag
                    self.in_tag_matrix = in_tag_matrix
                    
                    #in_tag = np.expand_dims(in_tag, axis=2)
                    in_tag_matrix = np.concatenate([in_tag, in_tag_matrix[:,0:-self.max_text_len]], 1)
                    #in_tag_matrix = np.reshape(in_tag_matrix,[-1,self.sequence_length * self.max_text_len])
                    
                    self.in_idx_matrix = in_idx_matrix
                    self.in_idx = in_idx
                    
                    
                    
                    in_time_diff_matrix = np.concatenate([np.reshape(np.array(self.in_time_diff),
                                                                     [-1, 1]), in_time_diff_matrix[:, 0:-1]], 1)

                    def flip_swap_zero(matrix_in, fillval=-1):
                        matrix_2d = []
                        for batch_cnt in range(matrix_in.shape[0]):
                            tmp_val = matrix_in[batch_cnt]
                            tmp_val = np.flip(tmp_val[np.nonzero(tmp_val - fillval)], 0)
                            matrix_2d.append(np.concatenate(
                                [tmp_val, np.array([fillval] * (len(matrix_in[batch_cnt]) - len(tmp_val)))], 0))
                        return matrix_2d
                    
                    #self.test_in_idx_matrix = in_idx_matrix
                    #self.test_in_idx = in_idx
                    
                    
                    in_idx_matrix_swap = flip_swap_zero(in_idx_matrix)
                    in_tag_matrix_swap = flip_swap_zero(in_tag_matrix)
                    
                    in_time_diff_matrix_swap = flip_swap_zero(in_time_diff_matrix, fillval=28)

                   
                    
                    num_to_select = int(len(in_sessionId) * 9.0 / 10)
                    
                    
                    train_set_index = list(np.arange(num_to_select))
                    batch_size_train = len(train_set_index)
                    
                    
                    test_set_index = list(np.arange(num_to_select, len(in_sessionId)))
                    batch_size_test = len(test_set_index)
                    
                    
                
                    self.item_tmp = np.nan_to_num(in_idx_matrix_swap)[train_set_index]
#                     self.tag_tmp = np.nan_to_num(in_tag_matrix_swap)[train_set_index]
#                     print("pos_idx ",pos_idx)
#                     print("np.nan_to_num(in_idx_matrix_swap)[train_set_index].shape ", np.nan_to_num(in_idx_matrix_swap)[train_set_index].shape)
#                     print("in_tag.shape ", np.asarray(in_tag).shape)
#                     print("in_tag_list.shape ", in_tag_list.shape)
#                     print("np.nan_to_num(in_tag_list).shape ", np.nan_to_num(in_tag_list).shape)
#                     print("np.nan_to_num(in_tag_list)[train_set_index].shape ", np.nan_to_num(in_tag_list)[train_set_index].shape)
                        
                    #print("np.nan_to_num(np.array(in_um_tip_id))[train_set_index]", np.nan_to_num(np.array(in_um_tip_id))[train_set_index].shape)
                    #print("np.nan_to_num(in_tag_matrix_swap)[train_set_index]", np.nan_to_num(in_tag_matrix_swap)[train_set_index].shape)
                    
                    feed_dict = {self.input_item_id_tensor: np.nan_to_num(in_idx_matrix_swap)[train_set_index], 
                                 self.input_tag_id_tensor: np.nan_to_num(in_tag_matrix_swap)[train_set_index],
                                 self.X_time_diff: np.nan_to_num(in_time_diff_matrix_swap)[train_set_index], 
                                 self.target_item_id_tensor: np.nan_to_num(np.array(out_idx))[train_set_index], 
                                 self.UM_tip_id_tensor: np.nan_to_num(np.array(in_um_tip_id))[train_set_index],
                                 self.batch_size: batch_size_train}
                    fetches = [self.yhat, self.cost, self.final_state, self.global_step, self.lr, self.train_op]
                    #                     fetches = [self.trfm_out]
                    
                    

                    train_global_step += 1
                    yhat, cost, state, step, lr, _ = self.sess.run(fetches, feed_dict)
                    #                     self.trfm_out_check = self.sess.run(fetches, feed_dict)
                    #                     print(self.trfm_out_check)

                    #                     return
                    

                    # evaluate training result
                    fetches = [self.yhat, self.cost]
                    yhat_train, cost_train = self.sess.run(fetches, feed_dict)


                    
                    feed_dict_test = {self.input_item_id_tensor: np.nan_to_num(in_idx_matrix_swap)[test_set_index],
                                      self.input_tag_id_tensor: np.nan_to_num(in_tag_matrix_swap)[test_set_index],
                                      self.X_time_diff: np.nan_to_num(in_time_diff_matrix_swap)[test_set_index], 
                                      self.target_item_id_tensor: np.nan_to_num(np.array(out_idx))[test_set_index],
                                      self.UM_tip_id_tensor: np.nan_to_num(np.array(in_um_tip_id))[test_set_index],
                                      self.batch_size: batch_size_test}

                    fetches_test = [self.yhat, self.cost]
                    yhat_test, cost_test = self.sess.run(fetches_test, feed_dict_test)
                    
                    

                    if np.isnan(cost):
                        print(str(epoch) + ':Nan error!')
                        self.error_during_train = True
                        return

                    cost_train_list.append(cost_train)

                    label_train = np.eye(yhat_train.shape[0])
                    self.yhat_train = yhat_train
                    self.label_train = label_train
                    label_1d_train = np.reshape(label_train, [-1])
                    prob_1d_train = np.reshape(yhat_train, [-1])
                    fpr, tpr, thresholds = metrics.roc_curve(label_1d_train, prob_1d_train, pos_label=1)
                    auc_train = metrics.auc(fpr, tpr)

                    label_test = np.eye(yhat_test.shape[0])
                    label_1d_test = np.reshape(label_test, [-1])
                    prob_1d_test = np.reshape(yhat_test, [-1])
                    fpr, tpr, thresholds = metrics.roc_curve(label_1d_test, prob_1d_test, pos_label=1)
                    auc_test = metrics.auc(fpr, tpr)

                    auc_train_list.append(auc_train)
                    
                    
                    
                    if step % 20 == 0:
                        #cost_test, auc_test = self.test_data()
                        
                        auc_test_list.append(auc_test)
                        cost_test_list.append(cost_test)
                        
                        avgc = np.mean(cost_train_list)
                        avgc_test = np.mean(cost_test_list)
                        auc_train_avg = np.mean(auc_train_list)
                        auc_test_avg = np.mean(auc_test_list)

                        print('{} Ori_file{}, Epoch{},Step{},lr:{:.6f},loss:{:.5f},auc:{:.5f},lossTst:{:.5f},aucTst:{:.5f},MaxaucTst:{:.5f},tolerant:{:.1f}\n'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), self.ori_file, epoch, step, lr, avgc, auc_train_avg, avgc_test, auc_test_avg, max_test_auc, tolerant_time))

                        if step > 10:
                            cost_train_list = []
                            cost_test_list = []
                            auc_train_list = []
                            auc_test_list = []
                
                        with open(self.base_path + self.model_name + ".log", "a") as log_file:
                            log_file.write( '{} Ori_file{}, Epoch{},Step{},lr:{:.6f},loss:{:.5f},auc:{:.5f},lossTst:{:.5f},aucTst:{:.5f},MaxaucTst:{:.5f},tolerant:{:.1f}\n'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), self.ori_file, epoch, step, lr, avgc, auc_train_avg, avgc_test, auc_test_avg, max_test_auc, tolerant_time))
                            
                            if max_test_auc > auc_test_avg:
                                if epoch > 0:
                                    tolerant_time -= 1
                                if tolerant_time == 0:
                                    finished = True
                                    break
                            else:
                                max_test_auc = auc_test_avg
                                tolerant_time = self.tolerant_time
                                self.saver.save(self.sess, '{}/DUIEN-model'.format(self.checkpoint_dir), global_step=epoch)

                start = start + minlen - 1
                mask = np.arange(len(iters))[(end - start) < 2]  # idx of ended sessions
                for idx in mask:
                    maxiter += 1
                    if maxiter >= len(self.offset_sessions) - 1:
                        finished = True
                        break
                    iters[idx] = maxiter
                    start[idx] = self.offset_sessions[session_idx_arr[maxiter]]
                    end[idx] = self.offset_sessions[session_idx_arr[maxiter] + 1]

                if len(mask) and self.reset_after_session:
                    in_idx_matrix[mask, :] = 0
                    in_tag_matrix[mask, :] = 0
                  
                    in_time_diff_matrix[mask, :] = 28

            if np.isnan(avgc):
                print('Epoch {}: Nan error!'.format(epoch, avgc))
                with open(self.base_path + self.model_name + ".log", "a") as log_file:
                    log_file.write( '{} Epoch {}: Nan error!'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), epoch,
                                                         avgc) + "\n")
                self.error_during_train = True
                return
            self.saver.save(self.sess, '{}/DUIEN-model'.format(self.checkpoint_dir), global_step=epoch)
            with open(self.base_path + self.model_name + ".log", "a") as log_file:
                log_file.write('ckpt to : ' + self.checkpoint_dir)
        '''        
        cost_test, auc_test = self.test_data()
        cut_auc_test_list.append(auc_test)
        cut_cost_test_list.append(cost_test)
        cut_avgc = np.mean(cut_cost_test_list)
        cut_avgauc = np.mean(cut_auc_test_list)
        print('Ori_file{}, Epoch{},cutLossTst:{:.5f},cutAucTst:{:.5f}\n'.format(
                                    self.ori_file, epoch, cut_avgc, cut_avgauc))
        '''
        _softmax_W_topN = tf.slice(self.softmax_W, [0, 0], [self.predictTopN, self.rnn_size])
        _softmax_b_topN = tf.slice(self.softmax_b, [0], [self.predictTopN])

        self._logits = tf.matmul(self.user_final, _softmax_W_topN, transpose_b=True) + _softmax_b_topN
        # self.logits = tf.matmul(self.user_vec2, softmax_W, transpose_b=True) + softmax_b
        self._yhat = self.final_activation(self._logits)

        # ## savedModel ## #
        self._yhat = tf.identity(self._yhat, name="yhat")

        # topNum = tf.divide(tf.constant(1000), tf.max(tf.constant(1), outSeqNum))
        self._topk_prob_sample, self._topk_ind_sample = tf.nn.top_k(self._yhat, 10, name="topk")

        self._topk_ind_sample = self._topk_ind_sample - 1

        self._topk_doc_id = self.item_id_hash_table_reverse.lookup(tf.cast(self._topk_ind_sample, tf.int64))

        _topk_prob_sample_1d = tf.reshape(self._topk_prob_sample, [-1])
        _topk_ind_sample_1d = tf.reshape(self._topk_ind_sample, [-1])
        _topk_doc_id_1d = tf.reshape(self._topk_doc_id, [-1])

        self._topk_ind_val = tf.add(_topk_ind_sample_1d, tf.constant(0), name="topk_ind_val")
        self._topk_prob_val = tf.add(_topk_prob_sample_1d, tf.constant(0.0), name="topk_prob_val")
        self._topk_doc_id = tf.identity(_topk_doc_id_1d, name="topk_doc_id")

print("done")

done


In [8]:
#del DUIEN
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7, allow_growth=True)    # determines the fraction of the overall amount of memory that each visible GPU can be used

#with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:

In [9]:

tf.reset_default_graph()
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
args.neg_pos_sam_rat = 7

#DUIEN.loaddata()
#DUIEN.init_model()

In [10]:
DUIEN = DUIEN4Rec(sess, args)
DUIEN.preprocess()
DUIEN.loaddata()

loading data by pickle. 



foo:569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
foo:577: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
DUIEN.init_model()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

init1
init2
init3


In [13]:
DUIEN.n_epochs = 20
DUIEN.fit()

2021-06-04 12:49:16 Ori_filefoursquare_seq, Epoch0,Step20,lr:0.001000,loss:5.41343,auc:0.61943,lossTst:3.26509,aucTst:0.48822,MaxaucTst:0.00000,tolerant:40.0

2021-06-04 12:49:20 Ori_filefoursquare_seq, Epoch0,Step40,lr:0.001000,loss:5.32893,auc:0.64961,lossTst:3.31512,aucTst:0.49834,MaxaucTst:0.48822,tolerant:40.0

2021-06-04 12:49:25 Ori_filefoursquare_seq, Epoch0,Step60,lr:0.001000,loss:5.10906,auc:0.69681,lossTst:3.21497,aucTst:0.57337,MaxaucTst:0.49834,tolerant:40.0

2021-06-04 12:49:29 Ori_filefoursquare_seq, Epoch0,Step80,lr:0.001000,loss:4.99671,auc:0.70984,lossTst:3.21638,aucTst:0.56000,MaxaucTst:0.57337,tolerant:40.0

2021-06-04 12:49:33 Ori_filefoursquare_seq, Epoch1,Step100,lr:0.001000,loss:4.48266,auc:0.84431,lossTst:3.57201,aucTst:0.50822,MaxaucTst:0.57337,tolerant:40.0

2021-06-04 12:49:37 Ori_filefoursquare_seq, Epoch1,Step120,lr:0.001000,loss:3.76353,auc:0.90486,lossTst:3.41494,aucTst:0.61544,MaxaucTst:0.57337,tolerant:39.0

2021-06-04 12:49:42 Ori_filefoursquare_seq, 

In [15]:
DUIEN.test()

HR_10  0.3056768558951965
HR_5  0.26055312954876275
HR_1  0.14070839398350315
MRR_10 0.19217478031160684
